In [1]:
import random
import numpy as np

# Función para dividir y combinar matrices con mutación
def dividir_y_combinar_matrices_con_mutacion(padre, madre):
    hijo = []
    
    # Recorrer las filas de los padres simultáneamente
    for fila_padre, fila_madre in zip(padre, madre):
        nuevo_hijo_fila = []

        # Tomar los primeros 20 caracteres no guion del padre
        letras_encontradasP1 = 0
        for char in fila_padre:
            if letras_encontradasP1 < 20:  
                nuevo_hijo_fila.append(char)
                if char != '-':
                    letras_encontradasP1 += 1
                
        # Tomar los caracteres de la madre hasta tener 20 letras empezando en la posición de la letra 20
        letras_encontradasM2 = 0
        for char in fila_madre:
            if (letras_encontradasM2 <= 20) and (char != '-'): 
                letras_encontradasM2 += 1
            if (letras_encontradasM2 > 20) and (letras_encontradasM2 < 40):
                nuevo_hijo_fila.append(char)
                if char != '-' :   
                    letras_encontradasM2 += 1

        # Tomar los caracteres restantes del padre a partir de la posición 7
        letras_encontradasP3 = 0
        for char in fila_padre:
            if (letras_encontradasP3 < 40) and char != '-':
                letras_encontradasP3 += 1
            if (letras_encontradasP3 >= 40):
                nuevo_hijo_fila.append(char)
        
        # Agregar la nueva fila al hijo
        hijo.append(nuevo_hijo_fila)
    
    # Mutación: añadir dos veces el carácter "-" en lugares aleatorios
    for _ in range(2):
        # Elegir una fila aleatoria en la matriz hijo
        fila_mutacion = random.randint(0, len(hijo) - 1)
        
        # Encontrar una posición aleatoria donde insertar el "-"
        pos_insertar = random.randint(0, len(hijo[fila_mutacion]))
        
        # Insertar el "-" en la posición aleatoria
        hijo[fila_mutacion].insert(pos_insertar, '-')
    
    # Encontrar la longitud máxima de las filas en el hijo
    max_length = max(len(row) for row in hijo)
    
    # Ajustar todas las filas del hijo a la misma longitud máxima
    hijo_ajustado = []
    for row in hijo:
        row_padded = row + ['-'] * (max_length - len(row))
        hijo_ajustado.append(row_padded)
    
    # Convertir la lista de listas ajustada a una matriz
    hijo = np.array(hijo_ajustado)
    
    # Eliminar columnas conformadas solo por guiones
    columns_to_keep = np.any(hijo != '-', axis=0)
    hijo = hijo[:, columns_to_keep]
    
    return hijo

# Función para calcular el puntaje de una matriz
def calcular_puntaje(matriz):
    puntaje = 0
    # Recorrer las columnas de la matriz
    for col in matriz.T:
        char_count = {}
        # Contar la frecuencia de cada caracter en la columna
        for char in col:
            if char != '-':
                char_count[char] = char_count.get(char, 0) + 1

        # Evaluar el puntaje según las repeticiones de caracteres
        has_repeated_char_twice = any(count == 2 for count in char_count.values())
        has_repeated_char_three_times = any(count == 3 for count in char_count.values())

        if has_repeated_char_three_times:
            puntaje += 6
        elif has_repeated_char_twice:
            puntaje += 3
        else:
            puntaje -= 1
    return puntaje

# Entrada de usuario: tamaño de la población inicial y número de generaciones
poblacion_inicial = int(input("Introduce el tamaño de la población inicial: "))
num_generaciones = int(input("Introduce el número de generaciones a evolucionar: "))

# Secuencias de ADN
secuencia1 = "AATTACGTAGGCGCCCCGAGCGTTTATTCCTTAGTCCATTTACATTGGGATCGTGGTCTTGACCTCGAGATTTTGCAATTCTGATTAGGAGACGTCCTTA"
secuencia2 = "CTAGATTATCCCGACTCGTACTTGACGTTCACTCTCGCTCTACGCATGCTCTCGTAACTCGAAGTACTGGCGCTGTATACGACAAAGGGGGCTAGCCGGT"
secuencia3 = "GCGTTGTCTTTGATCTTAATGGCATCTCTGCCGCGCGCGCGGCGACTGGGCTTAGAGCCGGACTAACCTCCTTTTCCAAGCAGCTACCACGACCCCCGGA"

# Generar población inicial
poblacion_actual = []
for _ in range(poblacion_inicial):
    while True:
        # Insertar guiones aleatoriamente en las secuencias para crear una matriz
        lista_secuencia1 = list(secuencia1)
        lista_secuencia2 = list(secuencia2)
        lista_secuencia3 = list(secuencia3)

        total_chars = len(lista_secuencia1) + len(lista_secuencia2) + len(lista_secuencia3)
        indices = random.sample(range(total_chars), 3)

        for i in indices:
            if i < len(lista_secuencia1):
                lista_secuencia1.insert(i, '-')
            elif i < len(lista_secuencia1) + len(lista_secuencia2):
                lista_secuencia2.insert(i - len(lista_secuencia1), '-')
            else:
                lista_secuencia3.insert(i - len(lista_secuencia1) - len(lista_secuencia2), '-')

        max_length = max(len(lista_secuencia1), len(lista_secuencia2), len(lista_secuencia3))
        lista_secuencia1 += ['-'] * (max_length - len(lista_secuencia1))
        lista_secuencia2 += ['-'] * (max_length - len(lista_secuencia2))
        lista_secuencia3 += ['-'] * (max_length - len(lista_secuencia3))

        matriz = np.array([lista_secuencia1, lista_secuencia2, lista_secuencia3])
        columnas_guiones = np.all(matriz == '-', axis=0)

        if not any(columnas_guiones):
            break

    puntaje = calcular_puntaje(matriz)
    poblacion_actual.append((matriz, puntaje))

# Evolución a lo largo de varias generaciones
for generacion in range(1, num_generaciones + 1):
    nueva_poblacion = []

    # Ordenar la población actual por puntaje
    poblacion_actual.sort(key=lambda x: x[1])

    # Seleccionar la mitad superior de la población para la cruza
    poblacion_seleccionada = poblacion_actual[len(poblacion_actual) // 2:]

    # Mostrar individuos de la población actual
    print("=" * 50)
    print("=" * 50)
    print(f"{'-' * 20}Generación {generacion}{'-' * 20}")
    print("Individuos:")
    for i, (matriz, puntaje) in enumerate(poblacion_actual, start=1):
        print(f"Individuo {i}:")
        print(matriz)
        print("Puntaje:", puntaje)
        print()

    # Mostrar los individuos sobrevivientes antes de generar los hijos

    print(f"{'-' * 11}Sobrevivientes{'-' * 11}")
    for i, (matriz, puntaje) in enumerate(poblacion_seleccionada, start=1):
        print(f"Sobreviviente {i}:")
        print(matriz)
        print("Puntaje:", puntaje)
        print()

    # Cruzar y mutar la población seleccionada
    for i in range(0, len(poblacion_seleccionada), 2):
        if i + 1 < len(poblacion_seleccionada):  # Verificar si hay un par disponible
            padre1 = poblacion_seleccionada[i][0]
            madre1 = poblacion_seleccionada[i + 1][0]
            padre2 = poblacion_seleccionada[i][0]
            madre2 = poblacion_seleccionada[i + 1][0]
            
            hijo1 = dividir_y_combinar_matrices_con_mutacion(padre1, madre1)
            puntaje_hijo1 = calcular_puntaje(hijo1)
            
            hijo2 = dividir_y_combinar_matrices_con_mutacion(padre2, madre2)
            puntaje_hijo2 = calcular_puntaje(hijo2)
            
            nueva_poblacion.append((hijo1, puntaje_hijo1))
            nueva_poblacion.append((hijo2, puntaje_hijo2))

    # Combinar la población actual con la nueva población
    poblacion_actual = poblacion_actual[:len(poblacion_actual) // 2] + nueva_poblacion

    # Ordenar la población combinada por puntaje
    poblacion_actual.sort(key=lambda x: x[1])

    # Mostrar la población de la siguiente generación (incluyendo hijos)
    print(f"{'-' * 11}Hijos Generación {generacion}{'-' * 11}")
    for i, (hijo, puntaje_hijo) in enumerate(nueva_poblacion, start=1):
        print(f"Hijo {i}:")
        print(hijo)
        print("Puntaje del hijo:", puntaje_hijo)
        print()

    print("=" * 100)
    print("=" * 100)
    print("=" * 100)
    print("=" * 100)

--------------------Generación 1--------------------
Individuos:
Individuo 1:
[['A' 'A' 'T' 'T' 'A' 'C' 'G' 'T' 'A' 'G' 'G' 'C' 'G' 'C' 'C' 'C' 'C' 'G'
  'A' 'G' 'C' 'G' 'T' 'T' 'T' 'A' 'T' 'T' 'C' 'C' 'T' 'T' 'A' 'G' 'T' 'C'
  'C' 'A' 'T' 'T' 'T' 'A' 'C' 'A' 'T' 'T' 'G' 'G' 'G' 'A' 'T' 'C' 'G' 'T'
  'G' 'G' 'T' 'C' 'T' 'T' 'G' 'A' 'C' 'C' 'T' 'C' 'G' 'A' 'G' 'A' 'T' 'T'
  'T' 'T' 'G' 'C' 'A' 'A' 'T' 'T' 'C' 'T' 'G' 'A' 'T' 'T' 'A' 'G' '-' 'G'
  'A' 'G' 'A' 'C' 'G' 'T' 'C' 'C' 'T' 'T' 'A' '-']
 ['C' 'T' 'A' 'G' 'A' 'T' 'T' 'A' 'T' 'C' 'C' 'C' 'G' 'A' 'C' 'T' 'C' 'G'
  'T' 'A' 'C' 'T' 'T' 'G' 'A' 'C' 'G' 'T' 'T' 'C' 'A' 'C' 'T' 'C' 'T' 'C'
  'G' 'C' 'T' 'C' 'T' 'A' 'C' 'G' 'C' 'A' 'T' 'G' 'C' 'T' 'C' 'T' 'C' 'G'
  'T' 'A' 'A' 'C' 'T' 'C' 'G' 'A' 'A' 'G' 'T' 'A' 'C' 'T' 'G' 'G' 'C' 'G'
  'C' 'T' 'G' 'T' 'A' 'T' 'A' 'C' 'G' 'A' 'C' 'A' 'A' 'A' 'G' 'G' 'G' 'G'
  'G' 'C' 'T' 'A' 'G' 'C' 'C' 'G' 'G' 'T' '-' '-']
 ['G' 'C' 'G' 'T' 'T' 'G' 'T' 'C' 'T' '-' 'T' 'T' 'G' 'A' 'T' 'C' 'T' 'T'
  'A' 